In [ ]:
import random
import os, sys

import numpy as np
import torch
import timm
from torch.utils.data import Subset
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader

from sklearn.model_selection import StratifiedKFold

sys.path.append(os.path.abspath('..'))

# BaseLine 코드로 주어진 dataset.py model.py, loss.py를 Import 합니다.
from dataset import MaskBaseDataset, BaseAugmentation
from model import *
from loss import create_criterion

sys.path.append('../')

def seed_everything(seed):
    """
    동일한 조건으로 학습을 할 때, 동일한 결과를 얻기 위해 seed를 고정시킵니다.
    
    Args:
        seed: seed 정수값
    """
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
seed_everything(42)

In [ ]:
# -- parameters
img_root = '/opt/ml/input/data/train'

batch_size = 64
num_workers = 4
num_classes = 18

num_epochs = 100  # 학습할 epoch의 수
log_interval = 80

lr = 1e-4
lr_decay_step = 10
criterion_name = 'cross_entropy' # loss의 이름

train_log_interval = 20  # logging할 iteration의 주기
name = "02_model_results"  # 결과를 저장하는 폴더의 이름

# -- settings
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

In [ ]:
def getDataloader(dataset, train_idx, valid_idx, batch_size, num_workers):
    # 인자로 전달받은 dataset에서 train_idx에 해당하는 Subset 추출
    train_set = torch.utils.data.Subset(dataset,
                                        indices=train_idx)
    # 인자로 전달받은 dataset에서 valid_idx에 해당하는 Subset 추출
    val_set   = torch.utils.data.Subset(dataset,
                                        indices=valid_idx)
    
    # 추출된 Train Subset으로 DataLoader 생성
    train_loader = torch.utils.data.DataLoader(
        train_set,
        batch_size=batch_size,
        num_workers=num_workers,
        drop_last=True,
        shuffle=True
    )
    # 추출된 Valid Subset으로 DataLoader 생성
    val_loader = torch.utils.data.DataLoader(
        val_set,
        batch_size=batch_size,
        num_workers=num_workers,
        drop_last=True,
        shuffle=False
    )
    
    # 생성한 DataLoader 반환
    return train_loader, val_loader

In [ ]:
img_root = '/opt/ml/input/data/train/images'

dataset = MaskBaseDataset(img_root)

transform = BaseAugmentation(
    resize=[128, 96],
    mean=dataset.mean,
    std=dataset.std,
)

dataset.set_transform(transform)

In [ ]:
import pandas as pd
from torchvision.transforms import Resize, ToTensor, Normalize

from dataset import TestDataset

test_img_root = '/opt/ml/input/data/eval'   
# public, private 테스트셋이 존재하니 각각의 예측결과를 저장합니다.

# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_img_root, 'info.csv'))
image_dir = os.path.join(test_img_root, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
test_dataset = TestDataset(image_paths, resize=(128, 96))

test_loader = DataLoader(
    test_dataset,
    shuffle=False
)

In [ ]:
model = build_model(device)
model.load_state_dict(torch.load(f"ck/best.pth"))

In [ ]:
from tqdm import tqdm

with torch.no_grad():
    model.eval()
    labels = []
    for images in tqdm(test_loader):
        images = images.to(device)

        # Test Time Augmentation
        age_outs, gender_outs, mask_outs = model(images)
                
        age_preds = torch.argmax(age_outs, dim=-1).item()
        gender_preds = torch.argmax(gender_outs, dim=-1).item()
        mask_preds = torch.argmax(mask_outs, dim=-1).item()
        
        label = dataset.encode_multi_class(mask_preds, gender_preds, age_preds)
        labels.append(label)

submission['ans'] = labels
submission.to_csv('submission(15x5).csv', index=False)

print("submission save done")